# Neurons example, pt. 2: large datasets

In [1]:
import numpyro
from bayes_window import models, fake_spikes_explore, BayesWindow, BayesConditions, BayesRegression
from bayes_window.generative_models import generate_fake_spikes
import numpy as np
from importlib import reload
import altair as alt
alt.data_transformers.disable_max_rows()

try:
    alt.renderers.enable('altair_saver', fmts=['png'])
except Exception:
    pass

In [2]:

df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=20,
                                                                n_neurons=6,
                                                                n_mice=3,
                                                                dur=5,
                                                               mouse_response_slope=40,
                                                               overall_stim_response_strength=5)


## Step by step

### 1. Firing rate

In [4]:
bw1 = BayesConditions(df=df_monster, y='isi',treatment='stim', condition=['neuron_x_mouse','i_trial'], group='mouse')
bw1.fit(dist_y='gamma');
# bw.plot_model_quality()

2021-11-30 15:43:46.937780: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW


Untransformed dimension in ['neuron_x_mouse', 'i_trial'] may be a problem. If you made a new numpyro model, look in utils.rename_posterior() 
<xarray.DataArray 'slope' (chain: 4, neuron_x_mouse: 18, i_trial: 20, draw: 200)>
array([[[[ 2.05496639e-01, -1.23332642e-01,  5.72049767e-02, ...,
           1.35442659e-01,  1.94491595e-02, -4.14554775e-02],
         [ 2.65519530e-01, -1.15224615e-01,  3.47977281e-02, ...,
           1.28003508e-01, -1.37674987e-01,  1.38172805e-01],
         [-1.26889199e-02,  3.64163369e-01, -1.88080207e-01, ...,
           2.10027397e-02,  8.41769874e-02, -1.58533901e-02],
         ...,
         [-2.48875335e-01, -4.86549921e-03, -1.04419798e-01, ...,
          -2.27378830e-01,  1.76198035e-02, -3.14136118e-01],
         [-1.57199949e-01, -2.33469084e-01,  1.06595501e-01, ...,
          -2.59016573e-01,  5.60912713e-02, -1.21270530e-01],
         [ 1.20249018e-01, -1.07145742e-01,  1.90651089e-01, ...,
          -7.81020522e-02,  1.20727688e-01, -8.28569978e

### 2. Regression

In [5]:
bw2 = BayesRegression(df=bw1.posterior['mu_per_condition'],
                 y='center interval', treatment='stim', condition=['neuron_x_mouse'], group='mouse')
bw2.fit(model=models.model_hierarchical,
               do_make_change='subtract',
               dist_y='student',
               robust_slopes=False,
               add_group_intercept=False,
               add_group_slope=False,
               fold_change_index_cols=('stim', 'mouse', 'neuron_x_mouse'))

bw2.plot_model_quality()

AssertionError: 

In [ ]:
bw2.chart


## Each neuron separately via SVI
### 1. Firing rate

In [ ]:
from tqdm import tqdm

gb='neuron_x_mouse'
step1_res=[]
for i, df_m_n in tqdm(df_monster.groupby(gb)):
    bw1 = BayesConditions(df_m_n, y='isi',treatment='stim', condition=['i_trial'], group='mouse'
                     ).fit(dist_y='gamma',
#                                       fit_fn=fitting.fit_svi
                                     )
    posterior=bw1.posterior['mu_per_condition'].copy()
    posterior[gb] = i
    step1_res.append(posterior)

### 2. Regression
TODO add sigma to step 2 inputs


In [ ]:
import pandas as pd
bw2 = BayesRegression(pd.concat(step1_res),
                 y='center interval', treatment='stim', condition=['neuron_x_mouse'], group='mouse',
                  detail='i_trial')
bw2.fit(model=models.model_hierarchical,
        do_make_change='subtract',
        dist_y='student',
        robust_slopes=False,
        add_group_intercept=False,
        add_group_slope=False)
bw2.chart

## NUTS 1-step GLM

In [ ]:
# Gamma GLM
bw = BayesRegression(df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical,
       progress_bar=True,
       do_make_change='subtract',
       dist_y='gamma',
       robust_slopes=False,
       add_group_intercept=False,
       add_group_slope=False,
       fold_change_index_cols=('stim', 'mouse', 'neuron','neuron_x_mouse'))

bw.plot_model_quality()

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

## NUTS student

In [ ]:
bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical,
              progress_bar=True,
              do_make_change='subtract',
              dist_y='student',
              robust_slopes=False,
              add_group_intercept=False,
              add_group_slope=False,
              fold_change_index_cols=('stim', 'mouse', 'neuron','neuron_x_mouse'))

bw.plot_model_quality()

In [ ]:

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

In [ ]:

bw = BayesRegression(df_monster, y='isi', treatment='stim', condition='neuron_x_mouse', group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',
              progress_bar=True,
              dist_y='student',
              use_gpu=True,
              num_chains=1,
              num_warmup=500,
              add_group_slope=True, add_group_intercept=False,
              fold_change_index_cols=('stim', 'mouse', 'neuron','neuron_x_mouse'))

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)
bw.facet(column='mouse',width=200,height=200).display()

#bw.explore_models(use_gpu=True)

## NUTS Lognormal

In [ ]:
bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',
              progress_bar=True,
              use_gpu=True, num_chains=1, n_draws=1500, num_warmup=1500,
              dist_y='lognormal',
              add_group_slope=True, add_group_intercept=True,
              fold_change_index_cols=('stim', 'mouse', 'neuron'))

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

In [ ]:
bw.explore_models(add_group_slope=True)

## BarkerMH

In [ ]:
%%time
from bayes_window import fitting

from importlib import reload
reload(fitting)

bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',
              sampler=numpyro.infer.BarkerMH,
#               progress_bar=True,
              use_gpu=False, num_chains=1, n_draws=5000, num_warmup=3000,
              dist_y='student',
              add_group_slope=True, add_group_intercept=True,
              fold_change_index_cols=('stim', 'mouse', 'neuron'),
              fit_method=fitting.fit_numpyro,
             )

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

## Fit using SVI

In [ ]:
%%time
from bayes_window import fitting
bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',
              n_draws=5000,
              dist_y='gamma',
              add_group_slope=True, add_group_intercept=False,
              fold_change_index_cols=('stim', 'mouse', 'neuron'),
              fit_method=fitting.fit_svi,
             )

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

In [ ]:
%%time
from bayes_window import fitting
import numpyro
from importlib import reload
reload(fitting)
bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',

            autoguide=numpyro.infer.autoguide.AutoLaplaceApproximation,
            optim=numpyro.optim.Adam(step_size=0.0005),
            loss=numpyro.infer.Trace_ELBO(),
              dist_y='lognormal',
              add_group_slope=True, add_group_intercept=True,
              fold_change_index_cols=('stim', 'mouse', 'neuron'),
              fit_method=fitting.fit_svi,

              n_draws=int(1e5),
              num_warmup=int(1e5),
             )

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

In [ ]:
%%time
from bayes_window import fitting
import numpyro
#numpyro.enable_validation(False)
bw = BayesRegression(df=df_monster, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.model_hierarchical, do_make_change='subtract',
              #use_gpu=True,
            autoguide=numpyro.infer.autoguide.AutoLaplaceApproximation,
            optim=numpyro.optim.Adam(1),
            loss=numpyro.infer.Trace_ELBO(),
              dist_y='lognormal',
              add_group_slope=True, add_group_intercept=False,
              fold_change_index_cols=('stim', 'mouse', 'neuron'),
              fit_method=fitting.fit_svi,
              #progress_bar=False,
              n_draws=int(1e5),
              num_warmup=int(1e5),
             )

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

Pretty model

In [ ]:
reload(models)
bw = BayesRegression(df=df, y='isi', treatment='stim', condition=['neuron', 'mouse'], group='mouse')
bw.fit(model=models.reparam_model(models.model_hierarchical_for_render), do_make_change='subtract',
              progress_bar=True,
              use_gpu=False, num_chains=1, n_draws=1500, num_warmup=1500,
              dist_y='normal',
              add_group_slope=True, add_group_intercept=True,
              fold_change_index_cols=('stim', 'mouse', 'neuron'))

bw.plot(x='neuron', color='mouse', independent_axes=True, finalize=True)


bw.facet(column='mouse',width=200,height=200).display()

In [ ]:
#!pip install git+https://github.com/pyro-ppl/numpyro.git
from numpyro.contrib.render import render_model
reload(models)
render_model(models.model_hierarchical_for_render, model_args=(1, 1, 1, 1,
                                                    'gamma', True,
                                                    True),
             render_distributions=True)

In [ ]:
#!pip install git+https://github.com/pyro-ppl/numpyro.git
from numpyro.contrib.render import render_model
reload(models)
render_model(models.model_hierarchical_for_render, model_args=(1, 1, 1, 1,
                                                    'gamma', True,
                                                    False),
             render_distributions=True)

In [ ]:
#!pip install git+https://github.com/pyro-ppl/numpyro.git
from numpyro.contrib.render import render_model
reload(models)
render_model(models.model_hier_stim_one_codition, model_args=(1, 1, 1,
                                                    'gamma',
                                                    ),
             render_distributions=True)


n(Divergences) = 2


## Two-step

## Packaged version 1
Separate levels

In [3]:
    # bw = BayesRegression(df=df_monster, y='isi',treatment='stim', condition=['neuron_x_mouse'],
    #                           group='mouse', detail='i_trial')
    # bw=bw.fit_twostep_by_group(dist_y_step_one='gamma', dist_y='student')

    # bw.chart

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
2021-11-30 15:30:36.703976: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
[Parallel(n_jobs=12)]: Done  14 out of  18 | elapsed:  5.5min remaining:  1.6min
[Parallel(n_jobs=12)]: Done  18 out of  18 | elapsed:  6.3min finished


AssertionError: 

## Packaged version 2
No grouping in first step

In [3]:
# bw = BayesRegression(df=df_monster, y='isi',treatment='stim', condition=['neuron_x_mouse'], group='mouse', detail='i_trial')
# bw=bw.fit_twostep(dist_y_step_one='gamma', dist_y='student')

TypeError: __init__() missing 3 required positional arguments: 'df', 'y', and 'treatment'

In [ ]:
# bw.chart